### Text Selection Data Merge

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Result/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [9]:
#def multi_wordgroup_in_wordgroup(word_list, df_target, target_column):
#    source_wordgroup_list = word_list
#    df_target2 = df_target
#    target_column2 = target_column
#
#    searchlist = []
#    resultlist = []
#    manager = multiprocessing.Manager()
#    resultlist = manager.list()
#    searchlist = manager.list()
#
#    def word_in_wordgroup(wordgroup):        
#        try:
#            word_in_word_cluster = df_target2[df_target2[f"{target_column2}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
#            for result in word_in_word_cluster[f"{target_column2}"]:
#                searchlist.append(wordgroup)
#                resultlist.append(result)
#        except:
#            pass
#
#    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
#        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste
#
#    search_list = list(searchlist)
#    result_list = list(resultlist)
#    df_result = pd.DataFrame([search_list,result_list]).transpose()
#    df_result.columns = ["word",f"{target_column}"]
#    df_result
#
#    return df_result

#### Text Data Read

In [10]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
44995,hayırbu,871
44996,konferansına,871
44997,kardeşisin,871
44998,kaldırıyorsun,871


In [11]:
#df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected.csv")
#df_twogram_select = df_twogram_select.head(1000)
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
213666,olmadan gerçek,52
213667,epey kolay,52
213668,hafızan iyi,52
213669,senin yaşındaydım,52


In [12]:
#df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected.csv")
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
307731,arkadaş mı olmak,38
307732,olan birkaç şey,38
307733,da size anlatayım,38
307734,için olay yerine,38


In [13]:
#df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected.csv")
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
388274,söz veriyorum kurul iş,16
388275,çıkmak için can atıyor,16
388276,ve içinden alevlerin çıktığı,16
388277,ne zaman isterse ama,16


In [14]:
#df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected.csv")
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
122369,ile matrix arasında seçim yapmam,19
122370,hey o iyi bir adam,19
122371,beklemek için nasıl bir yer,19
122372,yani bak normalde beni bilirsin,19


In [15]:
#df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected.csv")
df_sixgram_select

,sixgram,frequency
0,sana söylemem gereken bir şey var,3646
1,bir iki üç dört beş altı,2025
2,söylemek istediğin bir şey var mı,1794
3,daha iyi bir fikrin var mı,1792
4,bunun iyi bir fikir olduğunu sanmıyorum,1743
...,...,...
300159,bu çocuğun bir süper babaya ihtiyacı,9
300160,sadece yaranın üzerine baskı uygulamaya çalıştım,9
300161,tamam bir iki hafta içinde uğra,9
300162,bana şükran gününde ananas toplama sözü,9


In [16]:
#df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected.csv")
df_sevengram_select

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
141654,akşam üçüncü ve muhtemelen son olarak yerde,9
141655,herifi durdurmak için sadece bir saat kırk,9
141656,görücü usulü evlenmek daha çok işe yarıyor,9
141657,konuda çok yanlış olduğundan kesinlikle eminim...,9


In [17]:
#df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected.csv")
df_eightgram_select

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
109469,ve bunu her yaptığında takım arkadaşlarından b...,7
109470,gizli gizli dolaşan genç bir haham imajı canla...,7
109471,için tek yol kahrolası paramı bana geri vermen,7
109472,teşkilatın görev kapsamında olsa bile görev se...,7


In [18]:
#df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected.csv")
df_ninegram_select

,ninegram,frequency
0,dokuz sekiz yedi altı beş dört üç iki bir,463
1,bir iki üç dört beş altı yedi sekiz dokuz,405
2,iki üç dört beş altı yedi sekiz dokuz on,287
3,on dokuz sekiz yedi altı beş dört üç iki,277
4,ve sen herkesin seni tanıdığı o yerde olmak is...,134
...,...,...
40393,artık sana kan nakli yapacak doktor hoffman da...,7
40394,senin ülken için potansiyel terörist teröristi...,7
40395,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7
40396,herkes benim iyi adam olduğuma inanmakta bu ka...,7


In [19]:
#df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected.csv")
df_tengram_select

,tengram,frequency
0,bir iki üç dört beş altı yedi sekiz dokuz on,287
1,on dokuz sekiz yedi altı beş dört üç iki bir,251
2,g i t a r i s y e n,128
3,t o x i c w o r l d,93
4,umutsuzca birinin arkasından koşuyorum daha gü...,76
...,...,...
30518,kendi ülken başkasının ülkesi ya da kendi kişi...,5
30519,son altı ay içinde aynı tarzda bir düzine soyg...,5
30520,benim hikayem birkaç yıl önce yayınlandı kendi...,5
30521,aslında ben bunun pek de kötü bir fikir olmadı...,5


#### Text Data Word-Word Group In Word Group

In [20]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

In [21]:
#multi_wordgroup_in_wordgroup(word_list, df_twogram_select, "twogram")

##### Word

In [22]:
# 1 Way Slow
#df_word_in_two = word_in_wordgroup(word_list, df_twogram_select, "twogram", simple=True)
#df_word_in_two

In [23]:
# 2 Way Fast
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "twogram"
df_target = df_twogram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_two

,word,twogram
0,yapıyordun,ne yapıyordun
1,yapıyordun,iş yapıyordun
2,yapıyordun,mı yapıyordun
3,yapıyordun,mi yapıyordun
4,yapıyordun,yapıyordun sen
...,...,...
426664,ayrıntısına,ayrıntısına kadar
426665,ayrıntısına,ince ayrıntısına
426666,kardeşisin,onun kardeşisin
426667,kardeşisin,kız kardeşisin


In [24]:
df_word_in_two["word"].nunique()

29823

In [25]:
df_word_in_two_list = df_word_in_two.groupby(["word"])["twogram"].apply(list).reset_index(name="twogram")
df_word_in_two_list

,word,twogram
0,a,"[l a, a evet, a j, a b, a aziz, and a, a ve, t..."
1,aa,"[aa evet, şey aa, aa bu, aa şey, aa öyle, aa h..."
2,aaa,[aaa şey]
3,aah,"[aah o, aah en]"
4,aang,"[aang adında, aang adlı]"
...,...,...
29818,şık,"[çok şık, şık bir, şık görünüyorsun, kadar şık..."
29819,şımarık,"[şımarık bir, şımarık ve, seni şımarık]"
29820,şınav,[şınav çek]
29821,şırınga,[bir şırınga]


In [26]:
#df_word_in_two_list.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)

In [27]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_three = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_three.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_three

,word,threegram
0,başkasının,sadece bir başkasının
1,başkasının,ya da başkasının
2,başkasının,ne de başkasının
3,başkasının,bir başkasının daha
4,başkasının,de bir başkasının
...,...,...
919657,ayrıntısına,ince ayrıntısına kadar
919658,ayrıntısına,ufak ayrıntısına kadar
919659,ayrıntısına,en ufak ayrıntısına
919660,kardeşisin,kardeşisin değil mi


In [28]:
df_word_in_three_list = df_word_in_three.groupby(["word"])["threegram"].apply(list).reset_index(name="threegram")
df_word_in_three_list

,word,threegram
0,a,"[a ve b, a b c, two and a, a r i, t a r, i t a..."
1,aah,"[aah o günkü, aah en uzun, aah o gün]"
2,aang,"[aang adında bir, aang adlı bir]"
3,ab,[kan grubu ab]
4,abartma,"[kadar da abartma, bu kadar abartma, o kadar a..."
...,...,...
24427,şüphesi,[kimsenin şüphesi yok]
24428,şüphesiz,"[hiç şüphesiz bu, hiç şüphesiz ki, şüphesiz ki..."
24429,şüpheye,[şüpheye yer bırakmayacak]
24430,şık,"[çok şık bir, şık bir yer, ne kadar şık, şık b..."


In [29]:
df_word_in_three["word"].nunique()

24432

In [30]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_four = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_four.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_four

,word,fourgram
0,güvenli,seni güvenli bir yere
1,güvenli,güvenli bir yere götür
2,güvenli,onu güvenli bir yere
3,güvenli,için güvenli bir yer
4,güvenli,daha güvenli bir yere
...,...,...
1544435,ayrıntısına,en ince ayrıntısına kadar
1544436,ayrıntısına,en ufak ayrıntısına kadar
1544437,ayrıntısına,şeyi en ince ayrıntısına
1544438,kardeşisin,de onun kız kardeşisin


In [31]:
df_word_in_four_list = df_word_in_four.groupby(["word"])["fourgram"].apply(list).reset_index(name="fourgram")
df_word_in_four_list

,word,fourgram
0,a,"[a b c d, i t a r, g i t a, a r i s, t a r i, ..."
1,aa,"[aa şey aa yani, ah aa bilirsin şey, aa yani t..."
2,aaa,"[aaa yoksa bu öyle, benimle aaa evlenir misin,..."
3,aaaah,[aaaah sadece bir saniyeliğine]
4,aaah,"[hiç aaah diye bağırıp, aaah diye bağırıp havaya]"
...,...,...
34299,şüphesiz,"[şüphesiz ki var bir, hiç şüphesiz bu bir, şüp..."
34300,şüpheye,"[en ufak bir şüpheye, parkın güvenliği hakkınd..."
34301,şüpheyle,[şüpheyle yaklaşıyorum tabii kendi]
34302,şık,"[ne kadar da şık, bu çok şık bir, burası çok ş..."


In [32]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_five

,word,fivegram
0,ifadesi,ifadesi olarak bu küçük hediyeyi
1,ifadesi,ve aptal bir yüz ifadesi
2,ifadesi,mart tarihinde alınan yeminli ifadesi
3,ifadesi,ifadesi belirsiz ve bir kanıtı
4,istemedi,kimse senden babayı oynamanı istemedi
...,...,...
606403,açılışını,aynı drama açılışını defalarca kez
606404,batıyoruz,batıyoruz ve gücümüz de yok
606405,ayrıntısına,şeyi en ince ayrıntısına kadar
606406,ayrıntısına,her şeyi en ince ayrıntısına


In [33]:
df_word_in_five_list = df_word_in_five.groupby(["word"])["fivegram"].apply(list).reset_index(name="fivegram")
df_word_in_five_list

,word,fivegram
0,a,"[take a woman like you, i t a r i, t a r i s, ..."
1,aa,"[şey aa yani tüm kasabaya, hiç de iyi değil bu..."
2,aaa,"[aaa yoksa bu öyle bir, aaa şey çünkü daha faz..."
3,aaah,"[hiç aaah diye bağırıp havaya, aaah diye bağır..."
4,aah,"[aah en uzun yolculuğuna daha, aah o gün olanl..."
...,...,...
27035,şüphesi,"[bundan hiç şüphesi oldu mu, hiç şüphesi oldu ..."
27036,şüphesiz,"[şüphesiz ki var bir nedeni, ben hemen dönerim..."
27037,şüpheye,"[parkın güvenliği hakkında şüpheye düştü, oldu..."
27038,şık,"[şık bir isim değil mi, burası çok şık bir res..."


In [34]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_six

In [ ]:
df_word_in_six_list = df_word_in_six.groupby(["word"])["sixgram"].apply(list).reset_index(name="sixgram")
df_word_in_six_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_seven

In [ ]:
df_word_in_seven_list = df_word_in_seven.groupby(["word"])["sevengram"].apply(list).reset_index(name="sevengram")
df_word_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_eight

In [ ]:
df_word_in_eight_list = df_word_in_eight.groupby(["word"])["eightgram"].apply(list).reset_index(name="eightgram")
df_word_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_nine

In [ ]:
df_word_in_nine_list = df_word_in_nine.groupby(["word"])["ninegram"].apply(list).reset_index(name="ninegram")
df_word_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_word_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_word_in_ten

In [ ]:
df_word_in_ten_list = df_word_in_ten.groupby(["word"])["tengram"].apply(list).reset_index(name="tengram")
df_word_in_ten_list

In [ ]:
dfs = [df_word_select, df_word_in_two_list, df_word_in_three_list, df_word_in_four_list, df_word_in_five_list, df_word_in_six_list, df_word_in_seven_list, df_word_in_eight_list, df_word_in_nine_list, df_word_in_ten_list]

In [ ]:
df_word_merge = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='left'), dfs)
df_word_merge

In [ ]:
#df_word_merge.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx", index=False)
df_word_merge.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_word_text = pd.read_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx")
df_word_text = pd.read_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv")

In [ ]:
df_word_merge2 = df_word_text[~(df_word_text["twogram"].isnull() & df_word_text["threegram"].isnull() & df_word_text["fourgram"].isnull() & df_word_text["fivegram"].isnull() & \
df_word_text["sixgram"].isnull() & df_word_text["sevengram"].isnull() & df_word_text["eightgram"].isnull() & df_word_text["ninegram"].isnull() & df_word_text["tengram"].isnull())]
df_word_merge2

In [ ]:
#df_word_merge2.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result3.xlsx", index=False)
df_word_merge2.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Result3.csv", index=False)

##### Twogram

In [ ]:
# 1 Way Slow
#df_two_in_three = word_in_wordgroup(twogram_list, df_threegram_select, "threegram", simple=True)
#df_two_in_three.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_three

In [ ]:
# 2 Way Fast
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_three = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_three.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_three

In [ ]:
df_two_in_three_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_three_list

In [ ]:
#df_two_in_four = word_in_wordgroup(twogram_list, df_fourgram_select, "fourgram", simple=True)
#df_two_in_four.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_four

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_four = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_four.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_four

In [ ]:
df_two_in_four_list = df_two_in_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_four_list

In [ ]:
#df_two_in_five = word_in_wordgroup(twogram_list, df_fivegram_select, "fivegram", simple=True)
#df_two_in_five.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_five

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_five

In [ ]:
df_two_in_five_list = df_two_in_five.groupby(["twogram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_two_in_five_list

In [ ]:
#df_two_in_six = word_in_wordgroup(twogram_list, df_sixgram_select, "sixgram", simple=True)
#df_two_in_six.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_six

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_six

In [ ]:
df_two_in_six_list = df_two_in_six.groupby(["twogram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_two_in_six_list

In [ ]:
#df_two_in_seven = word_in_wordgroup(twogram_list, df_sevengram_select, "sevengram", simple=True)
#df_two_in_seven.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_seven

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_seven

In [ ]:
df_two_in_seven_list = df_two_in_seven.groupby(["twogram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_two_in_seven_list

In [ ]:
#df_two_in_eight = word_in_wordgroup(twogram_list, df_eightgram_select, "eightgram", simple=True)
#df_two_in_eight.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_eight

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_eight

In [ ]:
df_two_in_eight_list = df_two_in_eight.groupby(["twogram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_two_in_eight_list

In [ ]:
#df_two_in_nine = word_in_wordgroup(twogram_list, df_ninegram_select, "ninegram", simple=True)
#df_two_in_nine.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_nine

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_nine

In [ ]:
df_two_in_nine_list = df_two_in_nine.groupby(["twogram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_two_in_nine_list

In [ ]:
#df_two_in_ten = word_in_wordgroup(twogram_list, df_tengram_select, "tengram", simple=True)
#df_two_in_ten.rename(columns={"word":"twogram"}, inplace=True)
#df_two_in_ten

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_two_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_two_in_ten

In [ ]:
df_two_in_ten_list = df_two_in_ten.groupby(["twogram"])["tengram"].apply(list).reset_index(name="tengram")
df_two_in_ten_list

In [ ]:
df_twogram_select

In [ ]:
df_two_in_three_list = pd.read_excel("Two_In_Three_List.xlsx")
df_two_in_three_list

In [ ]:
df_two_in_four_list = pd.read_excel("Two_In_Four_List.xlsx")
df_two_in_four_list

In [ ]:
df_two_in_five_list = pd.read_excel("Two_In_Five_List.xlsx")
df_two_in_five_list

In [ ]:
df_two_in_six_list = pd.read_excel("Two_In_Six_List.xlsx")
df_two_in_six_list

In [ ]:
df_two_in_seven_list = pd.read_excel("Two_In_Seven_List.xlsx")
df_two_in_seven_list

In [ ]:
df_two_in_eight_list = pd.read_excel("Two_In_Eight_List.xlsx")
df_two_in_eight_list

In [ ]:
df_two_in_nine_list = pd.read_excel("Two_In_Nine_List.xlsx")
df_two_in_nine_list

In [ ]:
df_two_in_ten_list = pd.read_excel("Two_In_Ten_List.xlsx")
df_two_in_ten_list

In [ ]:
dfs2 = [df_twogram_select, df_two_in_three_list, df_two_in_four_list, df_two_in_five_list, df_two_in_six_list, df_two_in_seven_list, df_two_in_eight_list, df_two_in_nine_list, df_two_in_ten_list]

In [ ]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs2)
df_twogram_merge

In [ ]:
df_twogram_merge.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_twogram_text = pd.read_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx")

In [ ]:
df_twogram_merge2 = df_twogram_text[~(df_twogram_text["threegram"].isnull() & df_twogram_text["fourgram"].isnull() & df_twogram_text["fivegram"].isnull() & df_twogram_text["sixgram"].isnull() & \
df_twogram_text["sevengram"].isnull() & df_twogram_text["eightgram"].isnull() & df_twogram_text["ninegram"].isnull() & df_twogram_text["tengram"].isnull())]
df_twogram_merge2

In [ ]:
df_twogram_merge2.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx", index=False)

##### Threegram

In [ ]:
#df_three_in_four = word_in_wordgroup(threegram_list, df_fourgram_select, "fourgram", simple=True)
#df_three_in_four.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_four

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_four = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_four.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_four

In [ ]:
df_three_in_four_list = df_three_in_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_four_list

In [ ]:
#df_three_in_five = word_in_wordgroup(threegram_list, df_fivegram_select, "fivegram", simple=True)
#df_three_in_five.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_five

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_five

In [ ]:
df_three_in_five_list = df_three_in_five.groupby(["threegram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_three_in_five_list

In [ ]:
#df_three_in_six = word_in_wordgroup(threegram_list, df_sixgram_select, "sixgram", simple=True)
#df_three_in_six.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_six

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_six

In [ ]:
df_three_in_six_list = df_three_in_six.groupby(["threegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_three_in_six_list

In [ ]:
#df_three_in_seven = word_in_wordgroup(threegram_list, df_sevengram_select, "sevengram", simple=True)
#df_three_in_seven.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_seven

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_seven

In [ ]:
df_three_in_seven_list = df_three_in_seven.groupby(["threegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_three_in_seven_list

In [ ]:
#df_three_in_eight = word_in_wordgroup(threegram_list, df_eightgram_select, "eightgram", simple=True)
#df_three_in_eight.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_eight

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_eight

In [ ]:
df_three_in_eight_list = df_three_in_eight.groupby(["threegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_three_in_eight_list

In [ ]:
#df_three_in_nine = word_in_wordgroup(threegram_list, df_ninegram_select, "ninegram", simple=True)
#df_three_in_nine.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_nine

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_nine

In [ ]:
df_three_in_nine_list = df_three_in_nine.groupby(["threegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_three_in_nine_list

In [ ]:
#df_three_in_ten = word_in_wordgroup(threegram_list, df_tengram_select, "tengram", simple=True)
#df_three_in_ten.rename(columns={"word":"threegram"}, inplace=True)
#df_three_in_ten

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_three_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_three_in_ten

In [ ]:
df_three_in_ten_list = df_three_in_ten.groupby(["threegram"])["tengram"].apply(list).reset_index(name="tengram")
df_three_in_ten_list

In [ ]:
dfs3 = [df_threegram_select, df_three_in_four_list, df_three_in_five_list, df_three_in_six_list, df_three_in_seven_list, df_three_in_eight_list, df_three_in_nine_list, df_three_in_ten_list]

In [ ]:
df_threegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['threegram'], how='left'), dfs3)
df_threegram_merge

In [ ]:
df_threegram_merge.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_threegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx")

In [ ]:
df_threegram_merge2 = df_threegram_text[~(df_threegram_text["fourgram"].isnull() & df_threegram_text["fivegram"].isnull() & df_threegram_text["sixgram"].isnull() & \
df_threegram_text["sevengram"].isnull() & df_threegram_text["eightgram"].isnull() & df_threegram_text["ninegram"].isnull() & df_threegram_text["tengram"].isnull())]
df_threegram_merge2

In [ ]:
df_threegram_merge2.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx", index=False)

##### Fourgram

In [ ]:
#df_four_in_five = word_in_wordgroup(fourgram_list, df_fivegram_select, "fivegram", simple=True)
#df_four_in_five.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_five

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_five = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_five.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_five

In [ ]:
df_four_in_five_list = df_four_in_five.groupby(["fourgram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_four_in_five_list

In [ ]:
#df_four_in_six = word_in_wordgroup(fourgram_list, df_sixgram_select, "sixgram", simple=True)
#df_four_in_six.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_six

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_six

In [ ]:
df_four_in_six_list = df_four_in_six.groupby(["fourgram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_four_in_six_list

In [ ]:
#df_four_in_seven = word_in_wordgroup(fourgram_list, df_sevengram_select, "sevengram", simple=True)
#df_four_in_seven.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_seven

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_seven

In [ ]:
df_four_in_seven_list = df_four_in_seven.groupby(["fourgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_four_in_seven_list

In [ ]:
#df_four_in_eight = word_in_wordgroup(fourgram_list, df_eightgram_select, "eightgram", simple=True)
#df_four_in_eight.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_eight

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_eight

In [ ]:
df_four_in_eight_list = df_four_in_eight.groupby(["fourgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_four_in_eight_list

In [ ]:
#df_four_in_nine = word_in_wordgroup(fourgram_list, df_ninegram_select, "ninegram", simple=True)
#df_four_in_nine.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_nine

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_nine

In [ ]:
df_four_in_nine_list = df_four_in_nine.groupby(["fourgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_four_in_nine_list

In [ ]:
#df_four_in_ten = word_in_wordgroup(fourgram_list, df_tengram_select, "tengram", simple=True)
#df_four_in_ten.rename(columns={"word":"fourgram"}, inplace=True)
#df_four_in_ten

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_four_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_four_in_ten

In [ ]:
df_four_in_ten_list = df_four_in_ten.groupby(["fourgram"])["tengram"].apply(list).reset_index(name="tengram")
df_four_in_ten_list

In [ ]:
dfs4 = [df_fourgram_select, df_four_in_five_list, df_four_in_six_list, df_four_in_seven_list, df_four_in_eight_list, df_four_in_nine_list, df_four_in_ten_list]

In [ ]:
df_fourgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fourgram'], how='left'), dfs4)
df_fourgram_merge

In [ ]:
df_fourgram_merge.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_fourgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx")

In [ ]:
df_fourgram_merge2 = df_fourgram_text[~(df_fourgram_text["fivegram"].isnull() & df_fourgram_text["sixgram"].isnull() & df_fourgram_text["sevengram"].isnull() & \
df_fourgram_text["eightgram"].isnull() & df_fourgram_text["ninegram"].isnull() & df_fourgram_text["tengram"].isnull())]
df_fourgram_merge2

In [ ]:
df_fourgram_merge2.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx", index=False)

##### Fivegram

In [ ]:
#df_five_in_six = word_in_wordgroup(fivegram_list, df_sixgram_select, "sixgram", simple=True)
#df_five_in_six.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_six

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_six = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_six.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_six

In [ ]:
df_five_in_six_list = df_five_in_six.groupby(["fivegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_five_in_six_list

In [ ]:
#df_five_in_seven = word_in_wordgroup(fivegram_list, df_sevengram_select, "sevengram", simple=True)
#df_five_in_seven.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_seven

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_seven

In [ ]:
df_five_in_seven_list = df_five_in_seven.groupby(["fivegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_five_in_seven_list

In [ ]:
#df_five_in_eight = word_in_wordgroup(fivegram_list, df_eightgram_select, "eightgram", simple=True)
#df_five_in_eight.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_eight

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_eight

In [ ]:
df_five_in_eight_list = df_five_in_eight.groupby(["fivegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_five_in_eight_list

In [ ]:
#df_five_in_nine = word_in_wordgroup(fivegram_list, df_ninegram_select, "ninegram", simple=True)
#df_five_in_nine.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_nine

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_nine

In [ ]:
df_five_in_nine_list = df_five_in_nine.groupby(["fivegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_five_in_nine_list

In [ ]:
#df_five_in_ten = word_in_wordgroup(fivegram_list, df_tengram_select, "tengram", simple=True)
#df_five_in_ten.rename(columns={"word":"fivegram"}, inplace=True)
#df_five_in_ten

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_five_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_five_in_ten

In [ ]:
df_five_in_ten_list = df_five_in_ten.groupby(["fivegram"])["tengram"].apply(list).reset_index(name="tengram")
df_five_in_ten_list

In [ ]:
dfs5 = [df_fivegram_select, df_five_in_six_list, df_five_in_seven_list, df_five_in_eight_list, df_five_in_nine_list, df_five_in_ten_list]

In [ ]:
df_fivegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fivegram'], how='left'), dfs5)
df_fivegram_merge

In [ ]:
df_fivegram_merge.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_fivegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx")

In [ ]:
df_fivegram_merge2 = df_fivegram_text[~(df_fivegram_text["sixgram"].isnull() & df_fivegram_text["sevengram"].isnull() & \
df_fivegram_text["eightgram"].isnull() & df_fivegram_text["ninegram"].isnull() & df_fivegram_text["tengram"].isnull())]
df_fivegram_merge2

In [ ]:
df_fivegram_merge2.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx", index=False)

##### Sixgram

In [ ]:
#df_six_in_seven = word_in_wordgroup(sixgram_list, df_sevengram_select, "sevengram", simple=True)
#df_six_in_seven.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_seven

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_seven = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_seven.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_seven

In [ ]:
df_six_in_seven_list = df_six_in_seven.groupby(["sixgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_six_in_seven_list

In [ ]:
#df_six_in_eight = word_in_wordgroup(sixgram_list, df_eightgram_select, "eightgram", simple=True)
#df_six_in_eight.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_eight

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_eight

In [ ]:
df_six_in_eight_list = df_six_in_eight.groupby(["sixgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_six_in_eight_list

In [ ]:
#df_six_in_nine = word_in_wordgroup(sixgram_list, df_ninegram_select, "ninegram", simple=True)
#df_six_in_nine.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_nine

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_nine

In [ ]:
df_six_in_nine_list = df_six_in_nine.groupby(["sixgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_six_in_nine_list

In [ ]:
#df_six_in_ten = word_in_wordgroup(sixgram_list, df_tengram_select, "tengram", simple=True)
#df_six_in_ten.rename(columns={"word":"sixgram"}, inplace=True)
#df_six_in_ten

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_six_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_six_in_ten

In [ ]:
df_six_in_ten_list = df_six_in_ten.groupby(["sixgram"])["tengram"].apply(list).reset_index(name="tengram")
df_six_in_ten_list

In [ ]:
dfs6 = [df_sixgram_select, df_six_in_seven_list, df_six_in_eight_list, df_six_in_nine_list, df_six_in_ten_list]

In [ ]:
df_sixgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sixgram'], how='left'), dfs6)
df_sixgram_merge

In [ ]:
df_sixgram_merge.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_sixgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx")

In [ ]:
df_sixgram_merge2 = df_sixgram_text[~(df_sixgram_text["sevengram"].isnull() & df_sixgram_text["eightgram"].isnull() & df_sixgram_text["ninegram"].isnull() & df_sixgram_text["tengram"].isnull())]
df_sixgram_merge2

In [ ]:
df_sixgram_merge2.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx", index=False)

##### Sevengram

In [ ]:
#df_seven_in_eight = word_in_wordgroup(sevengram_list, df_eightgram_select, "eightgram", simple=True)
#df_seven_in_eight.rename(columns={"word":"sevengram"}, inplace=True)
#df_seven_in_eight

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_eight = pd.DataFrame([search_list,result_list]).transpose()
df_seven_in_eight.columns = [f"{search_column_name}",f"{target_column}"]
df_seven_in_eight

In [ ]:
df_seven_in_eight_list = df_seven_in_eight.groupby(["sevengram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_seven_in_eight_list

In [ ]:
#df_seven_in_nine = word_in_wordgroup(sevengram_list, df_ninegram_select, "ninegram", simple=True)
#df_seven_in_nine.rename(columns={"word":"sevengram"}, inplace=True)
#df_seven_in_nine

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_seven_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_seven_in_nine

In [ ]:
df_seven_in_nine_list = df_seven_in_nine.groupby(["sevengram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_seven_in_nine_list

In [ ]:
#df_seven_in_ten = word_in_wordgroup(sevengram_list, df_tengram_select, "tengram", simple=True)
#df_seven_in_ten.rename(columns={"word":"sevengram"}, inplace=True)
#df_seven_in_ten

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_seven_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_seven_in_ten

In [ ]:
df_seven_in_ten_list = df_seven_in_ten.groupby(["sevengram"])["tengram"].apply(list).reset_index(name="tengram")
df_seven_in_ten_list

In [ ]:
dfs7 = [df_sevengram_select, df_seven_in_eight_list, df_seven_in_nine_list, df_seven_in_ten_list]

In [ ]:
df_sevengram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sevengram'], how='left'), dfs7)
df_sevengram_merge

In [ ]:
df_sevengram_merge.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_sevengram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx")

In [ ]:
df_sevengram_merge2 = df_sevengram_text[~(df_sevengram_text["eightgram"].isnull() & df_sevengram_text["ninegram"].isnull() & df_sevengram_text["tengram"].isnull())]
df_sevengram_merge2

In [ ]:
df_sevengram_merge2.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx", index=False)

##### Eightgram

In [ ]:
#df_eight_in_nine = word_in_wordgroup(eightgram_list, df_ninegram_select, "ninegram", simple=True)
#df_eight_in_nine.rename(columns={"word":"eightgram"}, inplace=True)
#df_eight_in_nine

In [ ]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_nine = pd.DataFrame([search_list,result_list]).transpose()
df_eight_in_nine.columns = [f"{search_column_name}",f"{target_column}"]
df_eight_in_nine

In [ ]:
df_eight_in_nine_list = df_eight_in_nine.groupby(["eightgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_eight_in_nine_list

In [ ]:
#df_eight_in_ten = word_in_wordgroup(eightgram_list, df_tengram_select, "tengram", simple=True)
#df_eight_in_ten.rename(columns={"word":"eightgram"}, inplace=True)
#df_eight_in_ten

In [ ]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_eight_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_eight_in_ten

In [ ]:
df_eight_in_ten_list = df_eight_in_ten.groupby(["eightgram"])["tengram"].apply(list).reset_index(name="tengram")
df_eight_in_ten_list

In [ ]:
dfs8 = [df_eightgram_select, df_eight_in_nine_list, df_eight_in_ten_list]

In [ ]:
df_eightgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['eightgram'], how='left'), dfs8)
df_eightgram_merge

In [ ]:
df_eightgram_merge.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_eightgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx")

In [ ]:
df_eightgram_merge2 = df_eightgram_text[~(df_eightgram_text["ninegram"].isnull() & df_eightgram_text["tengram"].isnull())]
df_eightgram_merge2

In [ ]:
df_eightgram_merge2.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx", index=False)

##### Ninegram

In [ ]:
#df_nine_in_ten = word_in_wordgroup(ninegram_list, df_tengram_select, "tengram", simple=True)
#df_nine_in_ten.rename(columns={"word":"ninegram"}, inplace=True)
#df_nine_in_ten

In [ ]:
# parameter
source_wordgroup_list = ninegram_list
search_column_name = "ninegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

search_list = list(searchlist)
result_list = list(resultlist)
df_nine_in_ten = pd.DataFrame([search_list,result_list]).transpose()
df_nine_in_ten.columns = [f"{search_column_name}",f"{target_column}"]
df_nine_in_ten

In [ ]:
df_nine_in_ten_list = df_nine_in_ten.groupby(["ninegram"])["tengram"].apply(list).reset_index(name="tengram")
df_nine_in_ten_list

In [ ]:
dfs9 = [df_ninegram_select, df_nine_in_ten_list]

In [ ]:
df_ninegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['ninegram'], how='left'), dfs9)
df_ninegram_merge

In [ ]:
df_ninegram_merge.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx", index=False)

In [ ]:
df_ninegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx")

In [ ]:
df_ninegram_merge2 = df_ninegram_text[~(df_ninegram_text["tengram"].isnull())]
df_ninegram_merge2

In [ ]:
df_ninegram_merge2.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 4

In [ ]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
word_list = df_word_select["word"].to_list()

In [ ]:
df_two_three = pd.read_excel("/home/kurubal/Downloads/Twogram Threegram.xlsx")
df_two_three

In [ ]:
word_count_result(df_two_three,["threegram"])

In [ ]:
df_two_three["twogram"].count()

In [ ]:
df_two_three["threegram"].count()

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_two_three, "twogram","threegram",threegram_sample)
df_two_in_three

In [ ]:
set_all_twogram = set(df_two_three["twogram"].dropna().to_list())
set_twogram_in_threegram = set(df_two_in_three["twogram"].to_list())

In [ ]:
set_all_twogram.difference(set_twogram_in_threegram)

In [ ]:
set_threegram_cover_twogram = set(df_two_in_three["threegram"].to_list())
df_selected_threegram = pd.DataFrame(set_threegram_cover_twogram,columns=["threegram"])
df_selected_threegram

In [ ]:
df_selected_threegram.to_excel("Threegram_Selected.xlsx", index=False)

In [ ]:
word_count_result(df_selected_threegram,["threegram"])

In [ ]:
df_threegram_word_usage_select = word_usage_result(word_list, df_selected_threegram, "threegram", word_use_num_min, word_use_num_max)
df_threegram_word_usage_select

In [ ]:
word_count_result(df_threegram_word_usage_select,["threegram"])